In [1]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-09 18:07:36--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.76MB/s    in 0.2s    

2020-08-09 18:07:36 (4.76 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [28]:
from pyspark import SparkFiles
# Load in kitchen reviews from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

kitchen_df = spark.read.option("header", "true").csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), inferSchema=True, sep="\t", timestampFormat="yyyy-mm-dd")

kitchen_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...|2015-01-31 00:08:00|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...| 

## Drop duplicates and incomplete rows

In [29]:
print(f"Initial number of records: {kitchen_df.count()}")
kitchen_df = kitchen_df.dropna()
print(f"Number of records after removing nulls: {kitchen_df.count()}")
kitchen_df = kitchen_df.dropDuplicates()
print(f"Final number of records: {kitchen_df.count()}")

Initial number of records: 4880466
Number of records after removing nulls: 4879961
Final number of records: 4879961


## Examine the schema

In [6]:
kitchen_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Create DataFrame to match review_id_table table



In [7]:
# use to_date to convert timestamp to date
from pyspark.sql.functions import to_date

review_id_df = kitchen_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df = review_id_df.withColumn("review_date", to_date("review_date"))

review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1TYLPZVR9E1VM|      20696|B002RSO2PW|     609523155| 2014-01-18|
|R3DQZTU1JKZBJ3|      23426|B001ULD0BE|     493936758| 2013-01-03|
|R3D05QEKX6VBMB|      23527|B004AGYMCW|     136901658| 2014-01-25|
|R2MNOL5DIXFCHK|      37605|B002QXOF7I|     109528283| 2014-01-09|
|R1NWTTLC4DHVQM|      41593|B000QRAMMS|     468505544| 2013-01-23|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [8]:
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



## Create DataFrame to match products table

In [9]:
# Due to PK constraint unique values have to be selected
# Aggregate and drop count column
products_df = kitchen_df.groupBy("product_id", "product_title").agg({"product_id": "count"}).select(["product_id", "product_title"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0029SVQ9K|Update Internatio...|
|B00COM0AX4|Arctic Chill Sili...|
|B003HS1ZTC|Little Snowie Ice...|
|B003NQE8B0|Hamilton Beach 59...|
|B008YS1YQO|George Foreman Grill|
+----------+--------------------+
only showing top 5 rows



In [10]:
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



## Create DataFrame to match customers table

In [18]:
from pyspark.sql.types import IntegerType

customers_df = kitchen_df.groupBy('customer_id').agg({'review_id': 'count'}) \
                    .withColumnRenamed('count(review_id)', 'customer_count')
customers_df = customers_df.withColumn('customer_count', customers_df['customer_count'].cast(IntegerType()))

customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    1026543|             1|
|    1308596|             1|
|    1707289|             1|
|    2698443|             1|
|    2698445|             1|
+-----------+--------------+
only showing top 5 rows



In [19]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



## Create DataFrame to match vine_table table

In [21]:
vine_df = kitchen_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1TYLPZVR9E1VM|          5|            0|          0|   N|
|R3DQZTU1JKZBJ3|          5|            0|          0|   N|
|R3D05QEKX6VBMB|          4|            0|          0|   N|
|R2MNOL5DIXFCHK|          5|            0|          0|   N|
|R1NWTTLC4DHVQM|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [22]:
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



## Write DataFrames to RDS

In [23]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/<database-name>"
config = {"user":"<insert user name>",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}
          

In [24]:
# Write DataFrame to k_review_id_table table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='k_review_id_table', mode=mode, properties=config)

In [25]:
# Write DataFrame to k_products table in RDS
products_df.write.jdbc(url=jdbc_url, table='k_products', mode=mode, properties=config)

In [26]:
# Write DataFrame to k_customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table='k_customers', mode=mode, properties=config)

In [27]:
# Write DataFrame to k_vine_table table in RDS
vine_df.write.jdbc(url=jdbc_url, table='k_vine_table', mode=mode, properties=config)